# Урок 6: Оптимизация под ограничения Colab

**Решение проблем бесплатного Colab:**

| Проблема | Решение |
|----------|----------|
| Ограничение памяти (12-15GB) | 4-bit квантизация + gradient_checkpointing |
| Таймаут сессии (12 часов) | Автосохранение чекпоинтов каждые 50 шагов |
| Медленная загрузка | Кэш датасета в Google Drive |
| Отключение GPU | Восстановление из чекпоинта через accelerate |

In [ ]:
from transformers import TrainerCallback

class ColabSafeCallback(TrainerCallback):
    """Сохраняет чекпоинт каждые 50 шагов — защита от обрыва сессии Colab."""
    def __init__(self, save_path="/content/drive/MyDrive/checkpoints"):
        self.save_path = save_path
    
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 50 == 0 and state.global_step > 0:
            model = kwargs.get("model")
            if model:
                path = f"{self.save_path}/step-{state.global_step}"
                model.save_pretrained(path)
                print(f"✅ Чекпоинт сохранён: {path}")

In [ ]:
# Использование с Trainer
print("""
trainer = Trainer(
    model=model,
    ...
    callbacks=[ColabSafeCallback()],
    gradient_checkpointing=True,  # экономия ~30% памяти
)
""")

In [ ]:
# Проверка GPU и памяти
import torch

if torch.cuda.is_available():
    props = torch.cuda.get_device_properties(0)
    mem_gb = props.total_memory / 1024**3
    print(f"GPU: {props.name}")
    print(f"Память: {mem_gb:.2f} GB")
    print(f"Рекомендация: 4-bit для моделей 7B+ при <16GB" if mem_gb < 16 else "Можно full fine-tune маленьких моделей")
else:
    print("⚠️ GPU не обнаружен. В Colab: Runtime → Change runtime type → GPU")